In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import joblib
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

In [ ]:
# rf_model = joblib.load("../models/pm25/random_forest_model.pkl")
xgb_model = joblib.load("../models/pm25/xgboost_model.pkl")

In [3]:
x_scaler = joblib.load("../scalers/pm25/x_scaler.pkl")
y_scaler = joblib.load("../scalers/pm25/y_scaler.pkl")

In [ ]:
latitude = 41.9981
longitude = 21.4254

url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,windspeed_10m"
response = requests.get(url)

if response.status_code != 200:
    raise Exception("Error fetching weather data")

data = response.json()

hourly_data = {
    "time": data["hourly"]["time"],  
    "temperature": data["hourly"]["temperature_2m"],
    "humidity": data["hourly"]["relative_humidity_2m"],
    "wind_speed": data["hourly"]["windspeed_10m"]
}

df = pd.DataFrame(hourly_data)

df["time"] = pd.to_datetime(df["time"])

current_time = datetime.now()
next_2_days = current_time + timedelta(days=2)

df_next_2_days = df[(df["time"] >= current_time) & (df["time"] <= next_2_days)].copy()

df_next_2_days.loc[:, "hour"] = df_next_2_days["time"].dt.hour
df_next_2_days.loc[:, "year"] = df_next_2_days["time"].dt.year
df_next_2_days.loc[:, "month"] = df_next_2_days["time"].dt.month
df_next_2_days.loc[:, "day"] = df_next_2_days["time"].dt.day


X = df_next_2_days[['hour', 'year', 'month', 'day', 'temperature', 'humidity', 'wind_speed']]
X = x_scaler.transform(X)

# pm2_5_rf_predictions = rf_model.predict(X)
pm2_5_xgb_predictions = xgb_model.predict(X)


In [5]:
pm2_5_pred = y_scaler.inverse_transform(pm2_5_xgb_predictions.reshape(-1, 1))

In [ ]:
print(json.dumps({"pm2_5_predictions": pm2_5_pred.tolist()}))

array([[22.347288],
       [25.205452],
       [25.098516],
       [26.296879],
       [27.034092],
       [27.45743 ],
       [27.44353 ],
       [20.967422],
       [20.827843],
       [20.759012],
       [18.57541 ],
       [17.151628],
       [17.201746],
       [21.133675],
       [16.23869 ],
       [16.428087],
       [16.853802],
       [14.236129],
       [15.338324],
       [16.52703 ],
       [15.0702  ],
       [17.077232],
       [13.301709],
       [17.322788],
       [17.378426],
       [19.530357],
       [26.791779],
       [41.336254],
       [47.039204],
       [42.525017],
       [39.684162],
       [32.71347 ],
       [32.574184],
       [35.31098 ],
       [30.70112 ],
       [28.188808],
       [23.015959],
       [29.413368],
       [31.54876 ],
       [31.63969 ],
       [36.10969 ],
       [33.861088],
       [34.588165],
       [29.347286],
       [27.050568],
       [28.551935],
       [26.156816],
       [21.307577]], dtype=float32)